In [26]:
import pandas as pd
import numpy as np
from pathlib import Path
import hashlib

In [27]:
!pip install faker
from faker import Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.9 MB/s eta 0:00:00


In [15]:
def preprocess_postings(input_path, output_path):

    df = pd.read_csv(input_path, nrows=5000)
    # Select only the columns we need based on SQL schema
    needed_columns = [
        'title',
        'description',
        'job_id',
        'company_id'
    ]

    # Filter columns
    df = df[needed_columns]

    # Clean data
    df = df.dropna(subset=['job_id', 'company_id'])  # Remove rows with missing critical data
    df['title'] = df['title'].str[:10]  # Truncate to VARCHAR(10)
    df['description'] = df['description'].str[:2048]  # Truncate to VARCHAR(2048)

    # Remove duplicates
    df = df.drop_duplicates(subset=['title', 'job_id', 'company_id'])

    df.insert(0, 'posting_id', range(1, len(df) + 1))

    df = df.rename(columns={
            'title':'posting_title',
            'description':'posting_description',
            'job_id':'job_id',
            'company_id':'company_id'
    })

    # Save to new CSV
    df.to_csv(output_path, index=False)

In [17]:
def preprocess_jobs(input_path, output_path):
    """
    Process jobs data according to SQL schema
    """
    df = pd.read_csv(input_path, nrows=5000)

    # Select/rename columns based on salary information in your CSV
    df = df[['job_id', 'max_salary', 'med_salary']]

    # Clean data
    df = df.dropna(subset=['job_id'])

    # Convert salary columns to float/real
    df['max_salary'] = pd.to_numeric(df['max_salary'], errors='coerce')
    df['med_salary'] = pd.to_numeric(df['med_salary'], errors='coerce')

    # Remove duplicates on job_id since it's a primary key
    df = df.drop_duplicates(subset=['job_id'])

    df.to_csv(output_path, index=False)
    print(f"\nProcessed {len(df)} jobs")
    print("\nSample of processed data:")
    print(df.head())

In [10]:
def preprocess_companies(input_path, output_path):
    """
    Preprocess the companies.csv file to match our SQL schema.
    """
    df = pd.read_csv(input_path)

    needed_columns = [
        'company_id',
        'name',
        'description'
    ]

    df = df[needed_columns]
    df['name'] = df['name'].str[:10]
    df['description'] = df['description'].str[:2048]
    df = df.dropna(subset=['company_id'])
    df = df.drop_duplicates(subset=['company_id'])

    df = df.rename(columns={
    'company_id': 'company_id',
    'name': 'company_name',
    'description': 'company_description'
    })

    df.to_csv(output_path, index=False)

In [ ]:
def preprocess_skills(input_path, output_path):
    """
    Preprocess the skills.csv file to match our SQL schema.
    """
    df = pd.read_csv(input_path)

    df['embedding'] = np.random.random(size=len(df))
    
    # Convert embeddings to string representation (as a single REAL number for SQL)
    # We'll use the sum of the vector components as a simple representation
    
    df['skill_abr'] = df['skill_abr'].str[:10]
    df['skill_name'] = df['skill_name'].str[:50]
    df = df.dropna()
    df = df.drop_duplicates(subset=['skill_abr'])

    df = df.rename(columns={
    'skill_abr': 'skill_abbr',
    'skill_name': 'skill_name',
    'embedding': 'embedding'
    })

    df.to_csv(output_path, index=False)

In [20]:
# Create output directory if it doesn't exist
output_dir = Path('preprocessed_data')
output_dir.mkdir(exist_ok=True)

# Process each file
preprocess_postings('postings.csv', output_dir / 'postings_processed.csv')
preprocess_companies('companies.csv', output_dir / 'companies_processed.csv')
preprocess_jobs('postings.csv', output_dir / 'jobs_processed.csv')
#preprocess_skills('skills.csv', output_dir / 'skills_processed.csv')

print("Preprocessing complete!")

<ipython-input-10-e3f5f1d9cdf2>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['name'].str[:10]



Processed 5000 jobs

Sample of processed data:
     job_id  max_salary  med_salary
0    921716        20.0         NaN
1   1829192        50.0         NaN
2  10998357     65000.0         NaN
3  23221523    175000.0         NaN
4  35982263     80000.0         NaN
Preprocessing complete!


In [28]:
def generate_user_data(n_users=1000, output_path='preprocessed_data/users.csv'):
    """
    Generate dummy user data and save to CSV.
    Uses Faker to create realistic usernames and hashed passwords.
    """
    fake = Faker()

    # Generate unique usernames
    usernames = set()
    while len(usernames) < n_users:
        # Mix of formats: johnsmith, john.smith, john_smith93, etc.
        username = fake.user_name()
        if len(username) <= 20:  # Respect VARCHAR(20)
            usernames.add(username)

    # Convert to list and generate corresponding data
    users_data = {
        'user_id': range(1, n_users + 1),
        'username': list(usernames),
        'password': [hashlib.md5(fake.password().encode()).hexdigest() for _ in range(n_users)]  # Using MD5 for example purposes
    }

    # Create DataFrame
    df = pd.DataFrame(users_data)

    # Create output directory if it doesn't exist
    Path(output_path).parent.mkdir(exist_ok=True)

    # Save to CSV
    df.to_csv(output_path, index=False)

In [31]:
def generate_user_skills(n_relations=2000, users_file='preprocessed_data/users.csv',
                        skills_file='skills.csv',
                        output_path='preprocessed_data/user_skills.csv'):
    """
    Generate dummy user_skills relations between users and skills.
    Each user will have 1-5 random skills.
    """
    # Read users and skills
    users_df = pd.read_csv(users_file)
    skills_df = pd.read_csv(skills_file)

    # Generate random relationships
    relations = []
    for user_id in users_df['user_id']:
        # Randomly assign 1-5 skills to each user
        n_skills = np.random.randint(1, 6)
        skills = np.random.choice(skills_df['skill_abr'], size=n_skills, replace=False)

        for skill in skills:
            relations.append({
                'user_id': user_id,
                'skill_abr': skill
            })

    # Create DataFrame and take only the first n_relations if specified
    df = pd.DataFrame(relations).head(n_relations)

    # Remove any duplicates to maintain primary key constraint
    df = df.drop_duplicates()

    # Save to CSV
    df.to_csv(output_path, index=False)

In [32]:
generate_user_data(n_users=1000)
generate_user_skills(n_relations=2000)